# HW4


## Парсинг информации

Парсинг производился с сайта https://stopgame.ru;
Данное задание разделено на две части: 
    1.Собраны все обзоры на игры, что представлены на данном сайте;
    2.Собраны все карточки игр, что представлены на данном сайте.

#### ЧАСТЬ 1

*Импорт библиотек*

In [3]:
import requests 
from bs4 import BeautifulSoup 
from time import sleep
import random
import lxml 
import re
from tqdm import tqdm
import csv

*В ячейке ниже представлен "частный случай" кода для нахождение нужной информации*

In [154]:
shema = 'https://stopgame.ru'

url ='https://stopgame.ru/review/p1?subsection=izumitelno'
response = requests.get(url)
response.encoding ='utf-8'
soup = BeautifulSoup(response.text, 'lxml')

# выбор по оценкам
grade_choice = soup.find('div', style='display: flex; justify-content: flex-end; margin-bottom: 24px;').find_all('a')
grade = [f'{shema}{link["href"]}' for link in grade_choice]
grade_play = [re.sub(r'https://stopgame.ru/review/', '', grade[i]) for i in range(len(grade))]

# пагинация
pagen_end = int(soup.find('div', class_='_container_1mcqg_1').find_all('a')[-3].text)
pagen_quantity = [f'https://stopgame.ru/review/p{page}?subsection=izumitelno' for page in range(1,pagen_end+1)]

# ссыллки со страницы
card_description = soup.find('div', id='w0').find('div', class_='_default-grid_1uulc_206').find_all('article')
card_description_ = [y.select_one('a') for y in card_description]
card = [f'{shema}{x["href"]}' for x in card_description_] 

# каждая карточа, из которой взяли: заголовок, подзаголовок, автора, дату публикации, кол-во комментов, просмотров, статью
url = 'https://stopgame.ru/show/133444/the_legend_of_heroes_trails_to_azure_review'
response = requests.get(url)
response.encoding ='utf-8'
soup = BeautifulSoup(response.text, 'lxml')

date = soup.find('div', class_='_top-info_5it24_479').select_one('span').text
comments = soup.find('div', class_='_top-info_5it24_479').find('a').text.strip()
views = soup.find('div', class_='_top-info_5it24_479').select('span')[2].text.strip()
header =soup.find('h1').text
header_header = soup.find('h2').text
author = soup.find('div', class_='_bottom-info_5it24_577').find('span').text
review = ' '.join([opisanie.text for opisanie in soup.find('div', class_='_content_5it24_8').find_all('p')])

*Ниже получены все ссылки со страницами всех оценочных категорий*

In [167]:
pagen_quantity_grade = list()

for gp in tqdm(grade_play):
    url =f'https://stopgame.ru/review/p1?subsection={gp}'
    response = requests.get(url)
    response.encoding ='utf-8'
    soup = BeautifulSoup(response.text, 'lxml')
    
    pagen_end = int(soup.find('div', class_='_container_1mcqg_1').find_all('a')[-3].text)
    pagen_quantity = [f'https://stopgame.ru/review/p{page}?subsection={gp}' for page in range(1,pagen_end+1)]
    pagen_quantity_grade.extend(pagen_quantity)
    sleep(random.random())

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.43it/s]


*Собраны все ссылки с карточками*

In [171]:
card_all = list()
for link_card in tqdm(pagen_quantity_grade):
    response = requests.get(link_card)
    response.encoding ='utf-8'
    soup = BeautifulSoup(response.text, 'lxml')
    card_description = soup.find('div', id='w0').find('div', class_='_default-grid_1uulc_206').find_all('article')
    card_description_ = [y.select_one('a') for y in card_description]
    card = [f'{shema}{x["href"]}' for x in card_description_] 
    card_all.extend(card)
    sleep(random.random())
print(card_all)

100%|████████████████████████████████████████████████████████████████████████████████| 135/135 [01:59<00:00,  1.13it/s]

['https://stopgame.ru/show/133444/the_legend_of_heroes_trails_to_azure_review', 'https://stopgame.ru/show/132177/season_a_letter_to_the_future_review', 'https://stopgame.ru/show/132156/kenshi_review', 'https://stopgame.ru/show/131566/marvel_s_midnight_suns_review', 'https://stopgame.ru/show/131121/mount_blade_ii_bannerlord_review', 'https://stopgame.ru/show/130715/the_legend_of_heroes_trails_from_zero_my_zrya_propustili', 'https://stopgame.ru/show/130566/return_to_monkey_island_review', 'https://stopgame.ru/show/129851/rollerdrome_review', 'https://stopgame.ru/show/129157/stray_review', 'https://stopgame.ru/show/128702/cuphead_the_delicious_last_course_review', 'https://stopgame.ru/show/128640/ai_the_somnium_files_nirvana_initiative_review', 'https://stopgame.ru/show/128541/neon_white_review', 'https://stopgame.ru/show/127133/citizen_sleeper_review', 'https://stopgame.ru/show/127021/the_stanley_parable_ultra_deluxe_review', 'https://stopgame.ru/show/126719/rogue_legacy_2_review', 'http

*Берем всю нужную информацию с каждой карточки*

In [190]:
date = list()
comments = list()
views = list()
header = list()
header_header = list()
author = list()
review = list()

for url_card in tqdm(card_all):
    response = requests.get(url_card)
    response.encoding ='utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    date.append(soup.find('div', class_='_top-info_5it24_479').select_one('span').text)
    comments.append(soup.find('div', class_='_top-info_5it24_479').find('a').text.strip())
    views.append(soup.find('div', class_='_top-info_5it24_479').select('span')[2].text.strip())
    header.append(soup.find('h1').text)
    header_header.append(soup.find('h2').text)
    try:
        a = soup.find('div', class_='_bottom-info_5it24_577').find('span').text
    except:
        a = ''
    author.append(a)
    review.append(' '.join([opisanie.text for opisanie in soup.find('div', class_='_content_5it24_8').find_all('p')]))

    sleep(random.random())

100%|██████████████████████████████████████████████████████████████████████████████| 2398/2398 [35:11<00:00,  1.14it/s]


Примечание: в некоторых обзорах автор не указан. Завернул в блок try - except, т.к. ничего лучше не придумал.

*Создаем формат.csv файл*

In [201]:
with open('res.csv', 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['Заголовок', 'Подзаголовок','Автор', 'Дата', 'Просмотров', 'Комментариев','Описание'])

*Записываем в файл полученную информацию*

In [202]:
for header, header_header, author, date, views, comments, review in zip(header, header_header, author, date, views, comments, review):
    flatten = header, header_header, author, date, views, comments, review

    file = open('res.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(file, delimiter=';')
    writer.writerow(flatten)
file.close()
print('Файл res.csv создан')

Файл res.csv создан


#### ЧАСТЬ 2

*Импорт библиотек*

In [37]:
import requests
from bs4 import BeautifulSoup
import lxml
import re
from tqdm import tqdm
from time import sleep
from random import random
import csv

*В ячейках ниже представлен "частный случай" кода для нахождение нужной информации*

In [13]:
shema = 'https://stopgame.ru'

url = 'https://stopgame.ru/games/catalog'
responce = requests.get(url)
responce.encoding = 'utf-8'
soup = BeautifulSoup(responce.text, 'lxml')

In [14]:
# пагинация
page_count = int(soup.find('div', class_='_container_1mcqg_1').find_all('a')[-3].text)
url_page = [f'https://stopgame.ru/games/catalog?p={page}' for page in range(1,page_count+1)]

In [15]:
# ссылки всех карточек с каждой страницы
link = soup.find('div', id = 'w0', class_='list-view').find('div', class_='_games-grid_v95ji_304').find_all('a')
url_card = [f'{shema}{x["href"]}' for x in link]

In [16]:
# информация с каждой карточки
url = 'https://stopgame.ru/game/god_of_war_ragnar_k'
responce = requests.get(url)
responce.encoding = 'utf-8'
soup = BeautifulSoup(responce.text, 'lxml')

*С карточки будем тянуть следующую информацию: название, разработчик, издатель, платформа, дата выхода, оценка пользователя, кол-во оценок, (фото карточки?).*

In [17]:
name = soup.find('h1').text

creator = soup.find('div', class_='_info-grid__value_sh7r2_200').text

publisher = soup.find('div', class_='_info-grid_sh7r2_186').find('a').text

pl_inf = soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[2].find_all('a')
platforms = ', '.join([x.text for x in pl_inf])

g_inf = soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[-2].find_all('a')
genre = ', '.join([x.text for x in g_inf])

date = soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[-1].text

user_grade = soup.find('div', class_='_ratings-container_sh7r2_228').find('div', class_='_users-rating_sh7r2_361 _users-rating--green_sh7r2_1').find('span').text

c_g = soup.find('div', class_='_ratings-container_sh7r2_228').find('span',class_='_users-rating__count_sh7r2_1').text
counter_grade = re.sub(r'[\D]+','', c_g)

#photo_cards = soup.find('div', class_='_image-container_sh7r2_31').find('picture')

*Получаем все ссылки с карточками игр на сайте*

In [18]:
all_url_card = list()
for page_n in tqdm(url_page):
    responce = requests.get(page_n)
    responce.encoding = 'utf-8'
    soup = BeautifulSoup(responce.text, 'lxml')
    link = soup.find('div', id = 'w0', class_='list-view').find('div', class_='_games-grid_v95ji_304').find_all('a')
    url_card = [f'{shema}{x["href"]}' for x in link]
    all_url_card.extend(url_card)
    sleep(random())
print(all_url_card)

100%|████████████████████████████████████████████████████████████████████████████████| 763/763 [13:37<00:00,  1.07s/it]

['https://stopgame.ru/game/touhou_shinkirou_hopeless_masquerade', 'https://stopgame.ru/game/ark_survival_ascended', 'https://stopgame.ru/game/f1_23', 'https://stopgame.ru/game/all_dogs_go_to_heaven_activity_center', 'https://stopgame.ru/game/hatsune_miku_project_diva', 'https://stopgame.ru/game/princess_maker_4', 'https://stopgame.ru/game/melty_blood', 'https://stopgame.ru/game/grandia_online', 'https://stopgame.ru/game/sakura_wars_3_is_paris_burning', 'https://stopgame.ru/game/stepmania', 'https://stopgame.ru/game/need_for_speed_nitro', 'https://stopgame.ru/game/higurashi_when_they_cry_hou', 'https://stopgame.ru/game/impossible_spell_card', 'https://stopgame.ru/game/touhou_blooming_chaos', 'https://stopgame.ru/game/2_touhou_blooming_chaos_2', 'https://stopgame.ru/game/mountain_of_faith', 'https://stopgame.ru/game/lotus_land_story', 'https://stopgame.ru/game/legacy_of_lunatic_kingdom', 'https://stopgame.ru/game/imperishable_night', 'https://stopgame.ru/game/immaterial_and_missing_power

*Собираем всю необхадимую информации с каждой карточки.*

In [19]:
name = list()
creator = list()
publisher = list()
platforms = list()
genre = list()
date = list()
user_grade = list()
counter_grade = list()

for card_game in tqdm(all_url_card):
    responce = requests.get(card_game)
    responce.encoding = 'utf-8'
    soup = BeautifulSoup(responce.text, 'lxml')
    
    try:
        name.append(soup.find('h1').text)
    except:
        name.append('')
    
    try:
        creator.append(soup.find('div', class_='_info-grid__value_sh7r2_200').text)
    except:
        creator.append('')
        
    try:    
        publisher.append(soup.find('div', class_='_info-grid_sh7r2_186').find('a').text)
    except:
        publisher.append('')
        
    try:
        pl_inf = soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[2].find_all('a')
        plaforms.append(', '.join([x.text for x in pl_inf]))
    except:
        platforms.append('')
    
    try:
        date.append(soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[-1].text)
    except:
        date.append('')
        
    try:
        g_inf = soup.find('div', class_='_info-container__top_sh7r2_1').find_all('div',class_='_info-grid__value_sh7r2_200')[-2].find_all('a')
        genre.append(', '.join([x.text for x in g_inf]))
    except:
        genre.append('')
        
    try:
        user_grade.append(soup.find('div', class_='_ratings-container_sh7r2_228').find('div', class_='_users-rating_sh7r2_361 _users-rating--green_sh7r2_1').find('span').text)
    except:
        user_grade.append('')
        
    try:
        c_g = soup.find('div', class_='_ratings-container_sh7r2_228').find('span',class_='_users-rating__count_sh7r2_1').text
        counter_grade.append(re.sub(r'[\D]+','', c_g))
    except:
        counter_grade.append('')
    
    sleep(random())

100%|██████████████████████████████████████████████████████████████████████████| 22873/22873 [5:01:48<00:00,  1.26it/s]


*Создаем файл для загрузки данных*

In [40]:
with open('res2.csv', 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['Игра', 'Разработчик','Издатель', 'Платформа', 'Жанр', 'Дата выхода','Оценка пользователей', 'Кол-во оценок','Ссылка'])

In [41]:
for p1, p2, p3, p4, p5, p6, p7, p8, p9 in zip(name, creator, publisher, platforms, genre, date, user_grade, counter_grade, all_url_card):
    flatten = p1, p2, p3, p4, p5, p6, p7, p8, p9

    file = open('res2.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(file, delimiter=';')
    writer.writerow(flatten)
file.close()
print('Файл res2.csv создан')

Файл res2.csv создан
